<a href="https://colab.research.google.com/github/Wayodeni/university/blob/main/Tasks/%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_Python_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №5. Введение в Python. ООП рефакторинг**

---




**Обучающийся:** *Свиридов Лев Георгиевич*  



---

## **Цель работы:**



Провести рефакторинг ранее созданного Python-пакета в предыдущей практической работе, преобразовав его функциональность в соответствии с принципами объектно-ориентированного программирования. Обновленный пакет должен включать классы и методы для преобразования координат между декартовой и сферической системами координат, а также для работы с файлами.



## **Задачи:**



1. Анализ существующего пакета и определение необходимых изменений для перехода на ООП-парадигму.

2. Создание новой структуры пакета с использованием классов и модулей, соответствующих принципам ООП.

3. Реализация классов и методов для преобразования координат между декартовой и сферической системами координат.

4. Реализация классов и методов для работы с файлами, обеспечивая удобный интерфейс для чтения и записи данных.

5. Создание файла `__main__.py` с консольным интерфейсом для взаимодействия с функциональностью пакета.

6. Тестирование и проверка работоспособности обновленного пакета на примерах, подтверждение корректности реализации.



## **Демонстрация результата:**



Вставьте код каждого из ваших модулей в соответствующие ячейки ниже.

**Строку, начинающуюся на %%writefile... стирать запрещено**

### **Содержимое модуля \_\_init__.py:**

In [ ]:
!mkdir geo_transform

mkdir: cannot create directory ‘geo_transform’: File exists


In [ ]:
%%writefile geo_transform/__init__.py

from .file_operations import CoordinateRepository
from .transformations import Coordinates
from .utils import Angle

Overwriting geo_transform/__init__.py


### **Содержимое модуля transformations.py:**

In [ ]:
%%writefile geo_transform/transformations.py

import math
from enum import Enum
from functools import reduce
from typing import Literal

__all__ = ["Coordinates"]


class CoordinatesType(str, Enum):
    SPHERICAL = "spherical"
    CARTESIAN = "cartesian"


class Coordinates(tuple):
    # https://en.wikipedia.org/wiki/N-sphere#Spherical_coordinates
    coordinate_type: CoordinatesType | str

    def __new__(
        cls,
        iterable,
        coordinate_type: Literal[
            CoordinatesType.CARTESIAN, CoordinatesType.SPHERICAL
        ] = CoordinatesType.CARTESIAN,
    ):
        obj = super().__new__(cls, iterable)
        obj.coordinate_type = coordinate_type
        return obj

    @property
    def radius(self):
        if self.coordinate_type == CoordinatesType.SPHERICAL:
            return self[0]
        return math.sqrt(sum(x**2 for x in self))

    def spherical(self) -> tuple[float, ...]:
        if self.coordinate_type == CoordinatesType.SPHERICAL:
            return self

        r = self.radius
        n = len(self)
        angles = []
        for i in range(n - 2):
            numerator = self[i]
            denominator = math.sqrt(sum(x**2 for x in self[i:]))
            angles.append(math.acos(numerator / denominator))
        angles.append(math.atan2(self[-1], self[-2]))
        return (r, *angles)

    # https://en.wikipedia.org/wiki/N-sphere#cite_note-4
    def cartesian(self) -> tuple[float, ...]:
        if self.coordinate_type == CoordinatesType.CARTESIAN:
            return self

        r = self[0]
        angles = self[1:]
        n = len(angles) + 1

        coords = []
        for i in range(n):
            if i == n - 1:
                value = r * math.prod(math.sin(angles[j]) for j in range(n - 1))
            else:
                value = (
                    r
                    * math.cos(angles[i])
                    * math.prod(math.sin(angles[j]) for j in range(i))
                )
            coords.append(value)
        return tuple(coords)

    def raw(self):
        return self


Overwriting geo_transform/transformations.py


### **Содержимое модуля utils.py:**

In [ ]:
%%writefile geo_transform/utils.py

import math

__all__ = ["Angle"]


class Angle(float):
    def __init__(self, value: int | float):
        self.value = float(value)

    @property
    def rad(self) -> float:
        return math.radians(self.value)

    @property
    def deg(self) -> float:
        return math.degrees(self.value)

    def raw(self):
        return self.value


Overwriting geo_transform/utils.py


### **Содержимое модуля file_operations.py:**

In [ ]:
%%writefile geo_transform/file_operations.py

import csv
import json
from typing import Generator

from .transformations import Coordinates, CoordinatesType

__all__ = ["CoordinateRepository"]

CSV_FILE_TYPE = "csv"
SUPPORTED_FILE_TYPES = (CSV_FILE_TYPE,)


class CoordinateRepository:

    def write(
        self,
        coords: Coordinates,
        filename: str = "output.csv",
        append: bool = True,
    ):
        with open(
            filename, mode="w" if not append else "a", encoding="utf-8", newline=""
        ) as f:
            w = csv.writer(f)
            w.writerow(coords)
            return f.name

    def read(
        self,
        filename: str = "input.csv",
        coordinate_type: str = CoordinatesType.CARTESIAN,
    ) -> Generator[Coordinates, None, None]:
        if "." not in filename:
            raise ValueError(
                f"Unable to determine filetype. Provide file extension. Supported filetypes: {SUPPORTED_FILE_TYPES}"
            )
        elif filename.endswith(CSV_FILE_TYPE):
            with open(filename, mode="r", encoding="utf-8") as f:
                r = csv.reader(f)
                for line in r:
                    yield Coordinates(
                        tuple(float(coord) for coord in line),
                        coordinate_type=coordinate_type,
                    )
        else:
            raise ValueError(f"Unknown file type: {filename.split(".")[-1]}")


Overwriting geo_transform/file_operations.py


### **Содержимое модуля \_\_main__.py:**

In [ ]:
%%writefile geo_transform/terminal_ui.py
%%pip install cutie

import os
from typing import Any, Callable, Iterable

import cutie
import file_operations


# COMPONENTS
class Button:
    """
    Класс кнопки. (Пункт в списке, который возможно выбрать нажатием Enter)
    При нажатии на кнопку выполняется метод press, который вызывает функцию
    действия, которая была передана при создании объекта кнопки.
    """

    def __init__(self, name: str, action: Callable = lambda: None):
        self.set_name(name)
        self._action = action

    def get_name(self) -> str:
        return self._name

    def set_name(self, name: str) -> None:
        self._name = name

    def press(self):
        self._action()


MAIN_MENU_BUTTON = Button("В главное меню", lambda: title_and_main_menu())


def validated_input(
    input_name: str,
    validators: Iterable[Callable[[str], None]] = [],
    on_sucessful_confirm: Callable = lambda: None,
) -> str:
    """
    Инпут с возможностью валидации.
    Отображает строку input_name и запрашивает ввод пользователя.
    Не возвращает введенное пользователем значение, пока каждый из
    валидаторов из списка validators не пройдет без ошибок.

    Args:
        input_name (str): Текст, отображаемый перед инпутом
        validators (Iterable[Callable[[str], None]]): Список функций-валидаторов вводимого значения
        on_sucessful_confirm (_type_, optional): Функция, выполняемая после успешного ввода. Defaults to lambda:None.

    Returns:
        str: Значение, которое ввел пользователь
    """
    while True:
        value = input(f"{input_name}: ")

        errors: list[str]
        errors = []
        for validator in validators:
            try:
                validator(value)
            except ValidationError as e:
                errors.append(str(e))

        if len(errors) > 0:
            [print(error) for error in errors]
        else:
            on_sucessful_confirm()
            return value


def select(
    buttons: list[Button] = [],
    non_selectable_buttons: list[Button] | None = None,
    on_render: Callable = lambda: os.system("cls||clear"),
) -> None:
    """
    Отображение списка из кнопок с возможностью отключить выбор кнопок, указав их
    в non_selectable_buttons.

    Args:
        buttons (list[Button]): Список кнопок для отображения
        non_selectable_buttons (list[Button] | None, optional): Кнопки, которые нельзя будет выбрать. Defaults to None.
    """
    on_render()
    button_names = [button.get_name() for button in buttons]
    if non_selectable_buttons is not None:
        non_selectable_button_names = [
            button.get_name() for button in non_selectable_buttons
        ]
        non_selectable_button_indices = [
            button_names.index(non_selectable_name)
            for non_selectable_name in non_selectable_button_names
        ]
        pressed_button_index = cutie.select(button_names, non_selectable_button_indices)
    else:
        pressed_button_index = cutie.select(button_names)
    buttons[pressed_button_index].press()


def main_menu() -> None:
    print("")
    print("Главное меню")
    print("─" * 30)


def title() -> None:
    os.system("cls||clear")
    print("╭" + "─".center(60, "─") + "╮")
    print("│" + " ".center(60, " ") + "│")
    print("│" + "Geo Transform".center(60, " ") + "│")
    print("│" + " ".center(60, " ") + "│")
    print("│" + "Утилита для работы с координатами".center(60, " ") + "│")
    print("│" + "Разработал Свиридов Лев. Группа 2025-ФГИиБ-1м".center(60, " ") + "│")
    print("│" + " ".center(60, " ") + "│")
    print("╰" + "─".center(60, "─") + "╯")


def title_and_main_menu() -> None:
    title()
    main_menu()


def confirmation_prompt(
    question: str,
    on_confirm: Callable = lambda: None,
    on_reject: Callable = lambda: None,
    confirm_text: str = "Да",
    reject_text: str = "Нет",
) -> Any:
    """
    Запрашивает у пользователя подтверждение действия, обозначенного в question.
    В зависимости от согласия или отказа пользователя возвращает значение функции
    из on_confirm или on_reject.

    Args:
        question (str): Описание действия для соглашения/отказа пользователя
        on_confirm (_type_, optional): Функция, выполняемая при согласии. Defaults to lambda:None.
        on_reject (_type_, optional): Функция, выполняемая при отклонении. Defaults to lambda:None.
        confirm_text (str, optional): Текст кнопки подтверждения. Defaults to "Да".
        reject_text (str, optional): Текст кнопки отклонения. Defaults to "Нет".

    Returns:
        Any: Значение функции из on_confirm или on_reject
    """
    if cutie.prompt_yes_or_no(
        question,
        yes_text=confirm_text,
        no_text=reject_text,
        char_prompt=False,
        default_is_yes=True,
    ):
        return on_confirm()
    else:
        return on_reject()


def coordinate_input_with_caption_after_success(afterinput_string: str = ""):
    os.system("cls||clear")
    return validated_input(
        "Введите координаты, разделенные пробелом",
        [
            list_elems_validator(elem_validators=[only_digit_validator]),
        ],
        lambda: print(afterinput_string) if afterinput_string != "" else None,
    ).split()


# END COMPONENTS


# ERRORS
class ValidationError(Exception):
    """
    Исключение, выбрасываемое при ошибке валидации
    """

    pass


# END ERRORS


# VALIDATORS
def not_empty(value: str) -> None:
    if len(value) == 0 or value.isspace():
        raise ValidationError("Значение не должно быть пустым.")


def list_len_validator(length: int = 3, sep: str = " ") -> Callable[[str], None]:
    def validate(value: str) -> None:
        if len(value.split(sep)) != length:
            raise ValidationError(
                f"Количество элементов должно равняться {length}. Сейчас {len(value.split(sep))}."
            )

    return validate


def list_elems_validator(
    sep: str = " ",
    elem_validators: Iterable[Callable[[str], None]] = [lambda _: None],
) -> Callable[[str], None]:
    def validate(value: str) -> None:
        errors: Iterable[str] = []
        for i, elem in enumerate(value.split(sep)):
            for validator in elem_validators:
                try:
                    validator(elem)
                except Exception as e:
                    errors.append(
                        f"Ошибка для элемента '{elem}' под номером {i}: {str(e)}"
                    )
        if len(errors):
            raise ValidationError("\n".join(errors))

    return validate


def only_digit_validator(value: str) -> None:
    is_num = True
    try:
        float(value)
    except ValueError:
        is_num = False

    if not is_num:
        raise ValidationError(f"Значение '{value}' не является числом.")


# END VALIDATORS


# ACTIONS

# END ACTIONS


Overwriting geo_transform/terminal_ui.py


In [ ]:
%%writefile geo_transform/__main__.py

import file_operations
import terminal_ui
import transformations
import utils

if __name__ == "__main__":
    coordinate_repo = file_operations.CoordinateRepository()
    terminal_ui.title()
    terminal_ui.main_menu()
    while True:
        terminal_ui.select(
            on_render=lambda: None,
            buttons=[
                terminal_ui.Button(
                    "Преобразование координат",
                    lambda: terminal_ui.select(
                        [
                            terminal_ui.Button(
                                "Перевод в сферические",
                                lambda: print(
                                    transformations.Coordinates(
                                        map(
                                            float,
                                            terminal_ui.coordinate_input_with_caption_after_success(
                                                "Сферические координаты: "
                                            ),
                                        ),
                                        coordinate_type=transformations.CoordinatesType.CARTESIAN,
                                    ).spherical()
                                ),
                            ),
                            terminal_ui.Button(
                                "Перевод в декартовы",
                                lambda: print(
                                    transformations.Coordinates(
                                        map(
                                            float,
                                            terminal_ui.coordinate_input_with_caption_after_success(
                                                "Декартовы координаты: "
                                            ),
                                        ),
                                        coordinate_type="spherical",
                                    ).cartesian()
                                ),
                            ),
                        ]
                    ),
                ),
                terminal_ui.Button(
                    "Работа с файлами",
                    lambda: terminal_ui.select(
                        [
                            terminal_ui.Button(
                                "Записать результаты в файл",
                                lambda: print(
                                    coordinate_repo.write(
                                        tuple(
                                            map(
                                                float,
                                                terminal_ui.coordinate_input_with_caption_after_success(),
                                            )
                                        ),  # type: ignore
                                        terminal_ui.confirmation_prompt(
                                            "Хотите переопределить имя выходного файла (по умолчанию - output.csv)",
                                            on_confirm=lambda: terminal_ui.validated_input(
                                                "Введите имя файла"
                                            ),
                                            on_reject=lambda: "output.csv",
                                        ),
                                        terminal_ui.confirmation_prompt(
                                            "Дописать в конец",
                                            on_confirm=lambda: True,
                                            on_reject=lambda: False,
                                        ),
                                    )
                                ),
                            ),
                            terminal_ui.Button(
                                "Считать результаты из файла",
                                lambda: print(
                                    list(
                                        coordinate_repo.read(
                                            terminal_ui.validated_input(
                                                "Введите имя файла",
                                                [terminal_ui.not_empty],
                                            )
                                        )
                                    )
                                ),
                            ),
                        ]
                    ),
                ),
                terminal_ui.Button(
                    "Утилиты",
                    lambda: terminal_ui.select(
                        [
                            terminal_ui.Button(
                                "Градусы в радианы",
                                lambda: print(
                                    "Радиан: ",
                                    utils.Angle(
                                        float(
                                            terminal_ui.validated_input(
                                                "Введите число градусов",
                                                [
                                                    terminal_ui.not_empty,
                                                    terminal_ui.only_digit_validator,
                                                ],
                                            )
                                        )
                                    ).rad,
                                ),
                            ),
                            terminal_ui.Button(
                                "Радианы в градусы",
                                lambda: print(
                                    "Градусов: ",
                                    utils.Angle(
                                        float(
                                            terminal_ui.validated_input(
                                                "Введите число радиан",
                                                [
                                                    terminal_ui.not_empty,
                                                    terminal_ui.only_digit_validator,
                                                ],
                                            )
                                        )
                                    ).deg,
                                ),
                            ),
                        ]
                    ),
                ),
            ],
        )


Overwriting geo_transform/__main__.py


### **Содержимое модуля main.py (с импортом пакета и тестированием функций из него):**

In [ ]:
import random

from geo_transform import file_operations
from geo_transform.transformations import Coordinates
from geo_transform.utils import *

repository = file_operations.CoordinateRepository()
rand_range = (0, 100)
for _ in range(10):
    repository.write(
        Coordinates(
            (
                random.randint(*rand_range),
                random.randint(*rand_range),
                random.randint(*rand_range),
            )
        )
    )
for coord in repository.read("output.csv"):
    print("=======================================================================")
    print(f"Считанные координаты в сыром виде: {coord}")
    print(f"Считанные координаты в декартовой системе: {coord.cartesian()}")
    print(f"Считанные координаты в сферической системе: {coord.spherical()}")


Считанные координаты в сыром виде: (7.0, 12.0, 68.0)
Считанные координаты в декартовой системе: (7.0, 12.0, 68.0)
Считанные координаты в сферической системе: (69.40461079784254, 1.4697666939471095, 1.396124127786657)
Считанные координаты в сыром виде: (18.0, 73.0, 8.0)
Считанные координаты в декартовой системе: (18.0, 73.0, 8.0)
Считанные координаты в сферической системе: (75.61084578286372, 1.3304272740260599, 0.10915346290887773)
Считанные координаты в сыром виде: (91.0, 18.0, 93.0)
Считанные координаты в декартовой системе: (91.0, 18.0, 93.0)
Считанные координаты в сферической системе: (131.35448222272433, 0.8054568643656255, 1.379611867197882)
Считанные координаты в сыром виде: (67.0, 68.0, 23.0)
Считанные координаты в декартовой системе: (67.0, 68.0, 23.0)
Считанные координаты в сферической системе: (98.1936861514018, 0.8198581277881071, 0.3261558122434531)
Считанные координаты в сыром виде: (97.0, 10.0, 77.0)
Считанные координаты в декартовой системе: (97.0, 10.0, 77.0)
Считанные